# Function for GPR Plot
## Example Parameter: Avg Leaf Nitrogen Concentration (LNC) vs Leaf Carbon Nitrogen Ratio (LCN)
#### Author: Sofia Ingersoll

This is the list of the top 10 most common variables:
GPP, NBP, TOTVEGC, TLAI, EFLX_LH_TOT, SOILWATER_10CM, QRUNOFF, FSR, FAREA_BURNED, SNOWDP

In [1]:
# moved the libraries that were here into ml_utils.py because they're essential
# xarray is required to run the utils import line
import xarray as xr

from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, WhiteKernel
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

In [2]:
# import libraries & data pre-processing functions from utils.py
from ml_utils import *

In [4]:
# Request an additional 10 cores of power for processing from the server
client = get_cluster("UCSB0021", cores = 40)

/glade/u/home/singersoll/.conda/envs/meds-py/lib/python3.11/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 36831 instead
  warnings.warn(


In [5]:
# apply peer2peer network communication across multiple devices
client.cluster

PBSCluster(6f781b6f, 'tcp://128.117.208.103:35731', workers=40, threads=40, memory=372.40 GiB)

Prepare the data for GPR, including selecting the target variable and the parameters as features.

Split the data into training and testing sets.

Define a grid of hyperparameters for the GPR model.

Perform grid search cross-validation to find the best hyperparameters.

Fit the GPR model using the best hyperparameters on the training data.

Generate predictions and confidence intervals for each variable using the trained model.

Plot the results, including the observed data, predicted values, and confidence intervals.

### 1st Attempt

Testing new utils funct

In [11]:
param_avg = params

In [13]:
param_avg

<xarray.Dataset>
Dimensions:              (member: 500)
Coordinates:
  * member               (member) object 'LHC0001' 'LHC0002' ... 'LHC0500'
Data variables: (12/32)
    FUN_fracfixers       (member) float64 0.7783 0.3193 0.8768 ... 0.2621 0.4221
    KCN                  (member) float64 0.358 0.5488 0.2108 ... 0.9123 0.8392
    a_fix                (member) float64 0.255 0.6302 0.6621 ... 0.5339 0.1443
    crit_dayl            (member) float64 0.4012 0.27 0.3534 ... 0.9974 0.6571
    d_max                (member) float64 0.7978 0.5361 0.1409 ... 0.9076 0.4009
    fff                  (member) float64 0.1301 0.6792 ... 0.9918 0.02798
    ...                   ...
    stem_leaf            (member) float64 0.1138 0.1273 0.25 ... 0.3328 0.347
    sucsat_sf            (member) float64 0.7982 0.3947 0.1681 ... 0.3526 0.6515
    theta_cj             (member) float64 0.03646 0.09374 ... 0.8429 0.5055
    tpu25ratio           (member) float64 0.3936 0.04108 0.8197 ... 0.716 0.2124
    tpuse_sf             (member) float64 0.07896 0.2756 0.8401 ... 0.353 0.3634
    wc2wjb0              (member) float64 0.6846 0.8785 0.9519 ... 0.3581 0.1567

In [7]:
da = subset_var_cluster('LNC')

In [8]:
# only runs once, then outputs an error -- add a cute message to shorten warning
var_avg = wrangle_var_cluster(da)

In [44]:
param_vals = param_avg.values
param_da = np.array(param_vals)

In [45]:
param_da

array(<bound method Mapping.values of <xarray.Dataset>
Dimensions:              (member: 500)
Coordinates:
  * member               (member) object 'LHC0001' 'LHC0002' ... 'LHC0500'
Data variables: (12/32)
    FUN_fracfixers       (member) float64 0.7783 0.3193 0.8768 ... 0.2621 0.4221
    KCN                  (member) float64 0.358 0.5488 0.2108 ... 0.9123 0.8392
    a_fix                (member) float64 0.255 0.6302 0.6621 ... 0.5339 0.1443
    crit_dayl            (member) float64 0.4012 0.27 0.3534 ... 0.9974 0.6571
    d_max                (member) float64 0.7978 0.5361 0.1409 ... 0.9076 0.4009
    fff                  (member) float64 0.1301 0.6792 ... 0.9918 0.02798
    ...                   ...
    stem_leaf            (member) float64 0.1138 0.1273 0.25 ... 0.3328 0.347
    sucsat_sf            (member) float64 0.7982 0.3947 0.1681 ... 0.3526 0.6515
    theta_cj             (member) float64 0.03646 0.09374 ... 0.8429 0.5055
    tpu25ratio           (member) float64 0.3936 0.04

In [46]:
param_reshaped = param_vals.reshape(500, 32)

AttributeError: 'function' object has no attribute 'reshape'

In [50]:
param_reshaped = param_avg.stack(features=("member", np.arange(33)))

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [26]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# ----      Split Data 90/10        ----
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# Reshaping the data for splitting
# Reshape into (500, 32) where each row corresponds to a member and each column corresponds to a feature
#param_reshaped = param_avg.stack(features=("member", list(np.arange(3))))
#param_reshaped = param_avg.stack(features=("member", tuple(np.arange(33))))  # Convert np.arange(32) to a tuple
#param_reshaped = param_avg.stack(features=("member", list(np.arange(3))))



x_train, x_test, y_train, y_test = train_test_split(param_reshaped,
                                                    var_avg.values.reshape(-1, 1),
                                                    test_size=0.1,
                                                    # setting a seed
                                                    random_state=0)

AttributeError: 'function' object has no attribute 'reshape'

In [ ]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# ----      Set up Tune Grid       ----
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# Define a grid of hyperparameters for the GPR model
tune_grid = {
    "kernel": [ConstantKernel(constant_value=3, constant_value_bounds=(1e-4, 1e4) * RBF(length_scale=1.0) + WhiteKernel(noise_level=1e-4)],
    # Add more kernel configurations as needed
}

In [ ]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# ----      Cross Validation        ----
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# Perform grid search cross-validation to find the best hyperparameters
gpr = GaussianProcessRegressor(n_restarts_optimizer=20)

grid_search = GridSearchCV(gpr, tune_grid, cv=5, scoring='neg_mean_squared_error')

grid_search.fit(x_train, y_train)


In [ ]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# ----      Collect Metrics         ----
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# Get the best hyperparameters
best_kernel = grid_search.best_params_['kernel']

In [ ]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# ----      Finalize Workflow       ----
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# Fit the GPR model using the best hyperparameters on the training data
gpr_best = GaussianProcessRegressor(kernel=best_kernel)
gpr_best.fit(x_train, y_train)

In [ ]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# ----         Fit Model            ----
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# Generate predictions and confidence intervals for each variable using the trained model
y_pred, sigma = gpr_best.predict(x_pred, return_std=True)
# standard 0:1, set constant
x_pred = np.linspace(0,1,20)

In [ ]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# ----        Evaluate Model         ----
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# get accuracy of testing prediction, rmse
# get accuracy of testing prediction, rmse
y_pred_test = gpr_best.predict(x_test)
rmse = mean_squared_error(y_test, y_pred_test, squared=False)

In [ ]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# ----  User Selected ML Plotting Funct   ----
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# function to plot a cluster for to build on for ml
def cluster_ml_plot(param_avg, var_avg):


    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    # ----         Fit Model            ----
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    # Generate predictions and confidence intervals for each variable using the trained model
   # y_pred, sigma = gpr_best.predict(x_pred, return_std=True)     # to be included once upper portion of workflow optimized
   
    # standard 0:1, set constant
    x_pred = np.linspace(0,1,20)

    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    # ----         Plot Model           ----
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    plt.figure(figsize=(10, 6))
    plt.scatter(x_test, y_test, color='#62c900ff', label='Observed data')
    plt.plot(x_pred, y_pred, color='#134611', label='GPR Prediction')
    plt.fill_between(x_test.flatten(),
                     y_pred - 1.96 * sigma, y_pred + 1.96 * sigma,
                     alpha=0.5,
                     color='#9d6b53',
                     label = '95% Confidence Interval')
    # Set plot labels and title
    plt.xlabel(f"Perturbed Parameter: ({param_avg:})")  # ideas for later: param_name = getattr(param_avg)
    plt.ylabel(f"Variable: ({var_avg:})")
    plt.title('Parameter Value Uncertainty Estimation')
    plt.legend()
    # Show the plot
    plt.show()

In [ ]:
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Split data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(param_avg.values,
                                                    var_avg.values, 
                                                    test_size=0.1, 
                                                    random_state=0)

# Instantiate and fit the Gaussian Process Regression model
gpr = GaussianProcessRegressor()
gpr.fit(x_train, y_train)

# Make predictions on the testing data
y_pred = gpr.predict(x_test)

# Evaluate the model using root mean squared error
rmse = mean_squared_error(y_test, y_pred, squared=False)

# Print the RMSE
print("Root Mean Squared Error:", rmse)

## Custom Kernel: too big for realistic dashboard deployment
all psuedocode for now

In [ ]:
from sklearn.gaussian_process.kernels import Kernel, ConstantKernel, RBF, Matern, WhiteKernel, DotProduct, RationalQuadratic, ExpSineSquared

class CustomKernel(Kernel):
    def __init__(self, constant_value=1.0, constant_value_bounds=(1e-3, 1e3),
                 length_scale=1.0, length_scale_bounds=(1e-3, 1e3),
                 nu=1.5, nu_bounds=(1.5, 2.5),
                 alpha=1e-10, alpha_bounds=(1e-12, 1e-2),
                 noise_level=1e-5, noise_level_bounds=(1e-8, 1e-3),
                 periodicity=1.0, periodicity_bounds=(1e-3, 1e3)):
        self.constant_value = constant_value
        self.constant_value_bounds = constant_value_bounds
        self.length_scale = length_scale
        self.length_scale_bounds = length_scale_bounds
        self.nu = nu
        self.nu_bounds = nu_bounds
        self.alpha = alpha
        self.alpha_bounds = alpha_bounds
        self.noise_level = noise_level
        self.noise_level_bounds = noise_level_bounds
        self.periodicity = periodicity
        self.periodicity_bounds = periodicity_bounds

    def __call__(self, X, Y=None, eval_gradient=False):
        # Constant Kernel
        constant_kernel = ConstantKernel(
            constant_value=self.constant_value,
            constant_value_bounds=self.constant_value_bounds
        )

        # Radial Basis Function (RBF) Kernel
        rbf_kernel = RBF(
            length_scale=self.length_scale,
            length_scale_bounds=self.length_scale_bounds
        )

        # Matern Kernel
        matern_kernel = Matern(
            length_scale=self.length_scale,
            length_scale_bounds=self.length_scale_bounds,
            nu=self.nu,
            nu_bounds=self.nu_bounds
        )

        # Automatic Relevance Determination (ARD) Kernel
        ard_kernel = RBF(
            length_scale=self.length_scale,
            length_scale_bounds=self.length_scale_bounds
        ) * DotProduct(sigma_0=self.alpha, sigma_0_bounds=self.alpha_bounds)

        # Noise Kernel
        noise_kernel = WhiteKernel(
            noise_level=self.noise_level,
            noise_level_bounds=self.noise_level_bounds
        )

        # Linear Kernel
        linear_kernel = DotProduct(sigma_0=0.0)

        # Periodic Kernel
        periodic_kernel = ExpSineSquared(
            length_scale=self.periodicity,
            periodicity=self.periodicity,
            length_scale_bounds=self.periodicity_bounds,
            periodicity_bounds=self.periodicity_bounds
        )

        kernels = [constant_kernel, rbf_kernel, matern_kernel, ard_kernel, noise_kernel, linear_kernel, periodic_kernel]

        kernel_expression = sum(kernel(X, Y, eval_gradient) for kernel in kernels)

        if eval_gradient:
            K, K_gradient = kernel_expression
            return K, K_gradient
        else:
            return kernel_expression


### This is an area that needs to be addressed this week. However, my focus is primarily on GPR to prepare for our meeting with Linnia the Post Doc

In [ ]:
var_avg = wrangle_var_cluster(da)

In [ ]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# ----      Split Data 90/10        ----
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
x_train, x_test, y_train, y_test = train_test_split(param_avg.values.reshape(500, 32),
                                                    var_avg.values.reshape(-1, 1),
                                                    test_size=0.1,
                                                    # setting a seed
                                                    random_state=0)

In [ ]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# ----  User Selected ML Plotting Funct   ----
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# function to plot a cluster for to build on for ml
def cluster_ml_plot(param_avg, var_avg):


    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    # ----         Fit Model            ----
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    # Generate predictions and confidence intervals for each variable using the trained model
   # y_pred, sigma = gpr_best.predict(x_pred, return_std=True)     # to be included once upper portion of workflow optimized
   
    # standard 0:1, set constant
    x_pred = np.linspace(0,1,20)

    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    # ----         Plot Model           ----
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    plt.figure(figsize=(10, 6))
    plt.scatter(x_test, y_test, color='#62c900ff', label='Observed data')
    plt.plot(x_pred, y_pred, color='#134611', label='GPR Prediction')
    plt.fill_between(x_test.flatten(),
                     y_pred - 1.96 * sigma, y_pred + 1.96 * sigma,
                     alpha=0.5,
                     color='#9d6b53',
                     label = '95% Confidence Interval')
    # Set plot labels and title
    plt.xlabel(f"Perturbed Parameter: ({param_avg:})")  # ideas for later: param_name = getattr(param_avg)
    plt.ylabel(f"Variable: ({var_avg:})")
    plt.title('Parameter Value Uncertainty Estimation')
    plt.legend()
    # Show the plot
    plt.show()

In [ ]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# ----     load data stored in casper     ----
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
#------- Parameter Data---------
# x variable data for plotting
df = pd.read_csv('/glade/campaign/asp/djk2120/PPEn11/csvs/lhc220926.txt',index_col=0)
# the only dimension here is the 'member' aka file index id [LCH0001-500]
# convert to data set
params = xr.Dataset(df)
# subset in parameter file
leafcn = params['leafcn']

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# ----    variable wrangling     ----
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# apply the bug fixing & dim. reduction functions 
# read in variable cluster
da = read_all_simulations('LNC')

# feb. ncar time bug
da = fix_time(da)

# convert xr.ds to xr.da
da = da['LNC']

# weight gridcell dim by global land area
da_global = weight_landarea_gridcells(da,landarea)

# weight time dim by days in month
da_global_ann = yearly_weighted_average(da_global)

# take global avg for param over year dimension
lnc = da_global_ann.mean(dim='year')

In [ ]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# ----      Split Data 90/10        ----
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
x_train, x_test, y_train, y_test = train_test_split(leafcn.values.reshape(-1, 1), lnc.values.reshape(-1, 1), test_size=0.1, random_state=0)

In [ ]:
for p in params.data_vars:
    print(p)

In [ ]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# ----      Set up Tune Grid       ----
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# Define a grid of hyperparameters for the GPR model
tune_grid = {
    "kernel": [1.0 * RBF(length_scale=1.0) + WhiteKernel(noise_level=1e-5)],
    # Add more kernel configurations as needed
}

In [ ]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# ----      Cross Validation        ----
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# Perform grid search cross-validation to find the best hyperparameters
gpr = GaussianProcessRegressor(n_restarts_optimizer=20)

grid_search = GridSearchCV(gpr, tune_grid, cv=5, scoring='neg_mean_squared_error')

grid_search.fit(x_train, y_train)


In [ ]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# ----      Collect Metrics         ----
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# Get the best hyperparameters
best_kernel = grid_search.best_params_['kernel']

In [ ]:
best_kernel

In [ ]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# ----      Finalize Workflow       ----
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# Fit the GPR model using the best hyperparameters on the training data
gpr_best = GaussianProcessRegressor(kernel=best_kernel)
gpr_best.fit(x_train, y_train)

In [ ]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# ----         Fit Model            ----
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# Generate predictions and confidence intervals for each variable using the trained model
y_pred, sigma = gpr_best.predict(x_pred, return_std=True)
# standard 0:1, set constant
x_pred = np.linspace(0,1,20)

In [ ]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# ----        Evaluate Model         ----
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# get accuracy of testing prediction, rmse

In [ ]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# ----        Visual Model Fit      ----
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# Plot the results

plt.figure(figsize=(10, 6))
plt.scatter(x_test, y_test, color='#62c900ff', label='Observed data')
plt.plot(x_pred, y_pred, color='#134611', label='GPR Prediction')
plt.fill_between(x_test.flatten(), y_pred - 1.96 * sigma, y_pred + 1.96 * sigma, alpha=0.5, color='#9d6b53', label = '95% Confidence Interval')
plt.xlabel('Perturbed Parameter: Leaf Carbon to Nitrogen Ratio')
plt.ylabel('Variable: Leaf Nitrogen Concentration')
plt.title('Gaussian Process Regression Uncertainty Estimation')
plt.legend()
plt.show()

### 2nd attempt

In [ ]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# ----      Set up Tune Grid         ----
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# Define a grid of hyperparameters for the GPR model
tune_grid = {
    "kernel": [1.0 * RBF(length_scale=1.0)],
    # Add more kernel configurations as needed
}

In [ ]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# ----      Cross Validation        ----
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# Perform grid search cross-validation to find the best hyperparameters
gpr = GaussianProcessRegressor(n_restarts_optimizer=20)

grid_search = GridSearchCV(gpr, tune_grid, cv=5, scoring='neg_mean_squared_error')

grid_search.fit(x_train, y_train)

In [ ]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# ----      Collect Metrics         ----
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# Get the best hyperparameters
best_kernel = grid_search.best_params_['kernel']

In [ ]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# ----         Fit Model            ----
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# Generate predictions and confidence intervals for each variable using the trained model
y_pred, sigma = gpr_best.predict(x_test, return_std=True)

In [ ]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# ----        Visual Model Fit      ----
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# Plot the results
plt.figure(figsize=(10, 6))
plt.scatter(x_test, y_test, color='#62c900ff', label='Observed data')
plt.plot(x_test, y_pred, color='#134611', label='GPR Prediction')
# applying z-score for 95% CI
plt.fill_between(x_test.flatten(), y_pred - 1.96 * sigma, y_pred + 1.96 * sigma, alpha=0.5, color='#9d6b53', label = '95% Confidence Interval')
plt.xlabel('Perturbed Parameter: Leaf Carbon to Nitrogen Ratio')
plt.ylabel('Variable: Leaf Nitrogen Concentration')
plt.title('Gaussian Process Regression Uncertainty Estimation')
plt.legend()
plt.show()

### 3rd attempt

In [ ]:
# for simplicity's sake, we're going to use the preloaded data rn
leafcn = params['leafcn']

def plot_gpr(param, var):
    '''describe the relationship between the selected variable 
    and parameter(s) over the selected period of time. output a
    best fit line of regression.'''
    # Plotting
    plt.scatter(x=param, y=var, color = '#62c900ff', alpha = 0.8)
    # Set plot labels and title
    plt.xlabel(param)
    plt.ylabel(var)
    plt.title('2005-2010 Global Average')

    # Set boundaries

    # Gaussian Process Regression
    kernel = C(1.0, (1e-2, 1e2)) * RBF(1.0, (1e-2, 1e2))
    gp = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=20)
    x = param.reshape(-1, 1)
    y = var
    gp.fit(x, y)
    x_pred = np.linspace(0, 1, 100).reshape(-1, 1)
    y_pred, sigma = gp.predict(x_pred, return_std=True)
    plt.plot(x_pred, y_pred, '#134611', label='Gaussian Process Regression')
    plt.fill_between(x_pred.flatten(), y_pred - sigma, y_pred + sigma, alpha=0.2, color='#9d6b53')

    # Show legend with confidence interval
    plt.legend(loc='upper right',  bbox_to_anchor=(1, 1), labels=['Data', 'Gaussian Process Regression', '95% Confidence Interval'])


    # Show the plot
    plt.show()

In [ ]:
plot_gpr(leafcn, lnc)

In [ ]:
# Example usage:
plot_gpr('leafcn', lnc)